# 2021 Day 18

https://adventofcode.com/2021/day/18

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json

In [2]:
inp = open('input-18.txt').read()

In [3]:
def flatten(x):
    x = str(x)
    x = re.findall(r'\[|\]|,|\d+', x)
    depth = 0
    a, d = [], []
    for c in x:
        oc = c
        if c == '[':
            depth += 1
        elif c == ']':
            depth -= 1
        elif c == ',':
            pass
        else:
            a.append(int(c))
            d.append(depth)
    return np.array(a), np.array(d)

In [4]:
def explode(elements, depths):
    elements, depths = elements.copy(), depths.copy()
    N = len(elements)
    assert len(depths) == N
    could_explode = np.where(depths > 4)[0]
    if not could_explode.size:
        return False, elements, depths
    i1, i2 = could_explode[:2]
    if i1 >= 1:
        elements[i1-1] += elements[i1]
    if i2 <= N - 2:
        elements[i2+1] += elements[i2]
    elements = np.r_[elements[:i1], 0, elements[i2+1:]]
    depths = np.r_[depths[:i1], depths[i1]-1, depths[i2+1:]]
    return True, elements, depths

In [5]:
def split(elements, depths):
    elements, depths = elements.copy(), depths.copy()
    N = len(elements)
    assert len(depths) == N
    could_split = np.where(elements >= 10)[0]
    if not could_split.size:
        return False, elements, depths
    i = could_split[0]
    value, depth = elements[i], depths[i] + 1
    a, mod = value // 2, value % 2
    b = a + mod
    elements = np.r_[elements[:i], a, b, elements[i+1:]]
    depths = np.r_[depths[:i], depth, depth, depths[i+1:]]
    return True, elements, depths

In [6]:
def magnitude(elements, depths):
    out = 0
    while len(elements) > 1:
        could_mag = np.where(depths == depths.max())[0]
        i1, i2 = could_mag[:2]
        e1, e2 = elements[i1], elements[i2]
        d = depths[i1] - 1
        elements = np.r_[elements[:i1], 3*e1 + 2*e2, elements[i2+1:]]
        depths = np.r_[depths[:i1], d, depths[i2+1:]]
    return elements[0]

In [22]:
def reduce(elements, depths):
    needs_reduce = True
    while needs_reduce:
        did_reduce, elements, depths = explode(elements, depths)
        if did_reduce:
            continue
        needs_reduce, elements, depths = split(elements, depths)
    return elements, depths

In [8]:
def add(e1, d1, e2, d2):
    return np.concatenate((e1, e2)), np.concatenate((d1+1, d2+1))

In [9]:
def findsum(lines):
    if isinstance(lines, str):
        lines = lines.strip().split('\n')
    elements_depths = [flatten(x) for x in lines]
    elements, depths = elements_depths[0]
    for (es, ds) in elements_depths[1:]:
        elements, depths = reduce(*add(elements, depths, es, ds))
    return elements, depths

In [10]:
test_ex1 = [[[[[9,8],1],2],3],4]
test_ex2 = [7,[6,[5,[4,[3,2]]]]]
test_ex3 = [[6,[5,[4,[3,2]]]],1]
test_ex4 = [[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]
test_ex5 = [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]

In [11]:
print(*explode(*flatten(test_ex1)))
print(*explode(*flatten(test_ex2)))
print(*explode(*flatten(test_ex3)))
print(*explode(*flatten(test_ex4)))
print(*explode(*flatten(test_ex5)))

True [0 9 2 3 4] [4 4 3 2 1]
True [7 6 5 7 0] [1 2 3 4 4]
True [6 5 7 0 3] [2 3 4 4 1]
True [3 2 8 0 9 5 4 3 2] [2 3 4 4 2 3 4 5 5]
True [3 2 8 0 9 5 7 0] [2 3 4 4 2 3 4 4]


In [12]:
test_sp1 = [[[[0,7],4],[15,[0,13]]],[1,1]]
test_sp2 = [[[[0,7],4],[[7,8],[0,13]]],[1,1]]

In [13]:
print(*split(*flatten(test_sp1)))
print(*split(*flatten(test_sp2)))

True [ 0  7  4  7  8  0 13  1  1] [4 4 3 4 4 4 4 2 2]
True [0 7 4 7 8 0 6 7 1 1] [4 4 3 4 4 4 5 5 2 2]


In [14]:
test_mag1 = [[1,2],[[3,4],5]]
test_mag2 = [[[[0,7],4],[[7,8],[6,0]]],[8,1]]
test_mag3 = [[[[1,1],[2,2]],[3,3]],[4,4]]
test_mag4 = [[[[3,0],[5,3]],[4,4]],[5,5]]
test_mag5 = [[[[5,0],[7,4]],[5,5]],[6,6]]
test_mag6 = [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]

In [15]:
print(magnitude(*flatten(test_mag1)))
print(magnitude(*flatten(test_mag2)))
print(magnitude(*flatten(test_mag3)))
print(magnitude(*flatten(test_mag4)))
print(magnitude(*flatten(test_mag5)))
print(magnitude(*flatten(test_mag6)))

143
1384
445
791
1137
3488


In [16]:
test_r1 = [[[[[4,3],4],4],[7,[[8,4],9]]] , [1,1]]

In [17]:
print(*reduce(*flatten(test_r1)))

after addition
after addition
after addition
[0 7 4 7 8 6 0 8 1] [4 4 3 4 4 4 4 2 2]


In [18]:
(e1, d1), (e2, d2) = flatten([1,2]), flatten([[3,4],5])
print(*add(e1, d1, e2, d2))

[1 2 3 4 5] [2 2 3 3 2]


In [19]:
test_numsx = """
[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]
"""

In [23]:
print(*findsum(test_numsx))

[8 7 7 7 8 6 7 7 0 7 6 6 8 7] [4 4 4 4 4 4 4 4 4 4 4 4 3 3]


In [24]:
test_full1 = """
[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
"""

In [25]:
magnitude(*findsum(test_full1))

4140

## Part 1

In [26]:
magnitude(*findsum(inp))

4137

## Part 2

In [27]:
%%time
lines = inp.strip().split('\n')
top_mag = 0
for (i,line1) in enumerate(lines):
    for (j,line2) in enumerate(lines):
        print(f'\r{i+1:3d} + {j+1:3d}', end='', flush=True)
        if i == j:
            continue
        mag = magnitude(*findsum([line1,line2]))
        top_mag = max(mag, top_mag)
print()
top_mag

100 + 100
CPU times: user 44.9 s, sys: 12.4 s, total: 57.3 s
Wall time: 53.7 s


4573